In [6]:
!pip install -r ./requirements.txt -q


[notice] A new release of pip is available: 23.3 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [7]:
!pip show langchain

Name: langchain
Version: 0.0.342
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /workspace/.pyenv_mirror/user/current/lib/python3.12/site-packages
Requires: aiohttp, anyio, dataclasses-json, jsonpatch, langchain-core, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


In [8]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(),override=True)

os.environ.get('PINECONE_API_KEY')

'f6fae977-8607-404c-8f93-d1f9cab4f17a'

## LLM Models (Wrappers): GPT3

In [9]:
from langchain.llms import OpenAI
llm = OpenAI(model_name='text-davinci-003', temperature=.7, max_tokens=512)
print(llm)

OpenAI
Params: {'model_name': 'text-davinci-003', 'temperature': 0.7, 'top_p': 1, 'frequency_penalty': 0, 'presence_penalty': 0, 'n': 1, 'logit_bias': {}, 'max_tokens': 512}


In [10]:
output = llm('explain quantum mechanics in one sentance')
print (output)



Quantum mechanics is the study of the behavior of matter and energy at the atomic and subatomic level.


In [11]:
print (llm.get_num_tokens('explain quantum mechanics in one sentance'))

8


In [12]:
output= llm.generate(['....is the capital of framce', "what is the formula for the area of a circle"])

In [13]:
print (output.generations)

[[Generation(text='\n\nParis is the capital of France.', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text='\n\nThe formula for the area of a circle is A = πr², where r is the radius of the circle.', generation_info={'finish_reason': 'stop', 'logprobs': None})]]


In [14]:
print (output.generations[1][0].text)



The formula for the area of a circle is A = πr², where r is the radius of the circle.


In [15]:
len(output.generations)

2

In [16]:
foroutput = llm.generate(["write an original tagline for a burger resturatint"] * 3)

In [17]:
for o in output.generations:
    print (o[0].text, end='')



Paris is the capital of France.

The formula for the area of a circle is A = πr², where r is the radius of the circle.

#### ChatModels: GPT-3.5-Turbo and GPT-4

TODO: https://platform.openai.com/docs/guides/text-generation/chat-completions-api
Read more about what the different roles are

In [18]:
from langchain.schema import(
    AIMessage,
    HumanMessage,
    SystemMessage
)

from langchain.chat_models import ChatOpenAI


In [19]:
chat = ChatOpenAI(model_name= 'gpt-3.5-turbo', temperature=0.5, max_tokens=1024)
messages = [
    SystemMessage(content="you are a physicist and only respond in german"),
    HumanMessage(content="explain quantum mechanics in one sentance")
]
output = chat(messages)

In [20]:
print(output.content)

Quantenmechanik beschreibt das Verhalten von Teilchen auf atomarer und subatomarer Ebene und ermöglicht es uns, ihre Eigenschaften und Wechselwirkungen zu verstehen.


### Prompt Templates

In [21]:
from langchain import PromptTemplate

In [22]:
template = '''You are an experianced virologist.
Write a few sentances about the following {virus} in {language}.'''

prompt= PromptTemplate(
    input_variables=['virus','language'],
    template=template
)

print (prompt)

input_variables=['language', 'virus'] template='You are an experianced virologist.\nWrite a few sentances about the following {virus} in {language}.'


In [23]:
from langchain.llms import OpenAI
llm = OpenAI(model_name='text-davinci-003', temperature=.7)

output = llm(prompt.format(virus="ebola",language='english'))
print (output)



Ebola is a serious and often fatal virus that is spread through contact with an infected person's blood or body fluids. Symptoms of Ebola can include fever, severe headache, muscle pain, weakness, diarrhea, vomiting, abdominal pain, and unexplained bleeding or bruising. There is currently no cure for Ebola, but treatments can help to improve the chances of survival.


#### Simple Chains

In [24]:
pip install openai==0.28.1  


[notice] A new release of pip is available: 23.3 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [26]:
from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain

llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=.5)

template = '''You are an experianced virologist.
Write a few sentances about the following {virus} in {language}.'''

prompt= PromptTemplate(
    input_variables=['virus','language'],
    template=template
)

chain = LLMChain(llm=llm,prompt=prompt)
output = chain.run({"virus": "HSV","language":"english"})

#  output = chain.run('HSV')

In [27]:
print(output)

HSV, or herpes simplex virus, is a common viral infection that affects humans. There are two types of HSV: HSV-1, which primarily causes oral herpes, and HSV-2, which is responsible for genital herpes. Both types can be transmitted through direct contact with infected individuals or through contact with their bodily fluids. HSV infections are characterized by recurring outbreaks of painful blisters or sores, which can be managed with antiviral medications. While there is no cure for HSV, proper medical care and safe sexual practices can help prevent transmission and reduce the frequency and severity of outbreaks.
